<h2>instructions: <br><b>Press "runn all" and go to the bottom to connect to GoogleDrive</b></h2>


<b> (1) Functions used to generate two-dimensional second-order ordinary differential equations and calculate their fixed points </b>
<br><i>(slowest part of the code!)</i>

In [1]:
import sys
import pickle

import numpy as np
from sympy.solvers import nonlinsolve, solve, solveset

from sympy.geometry import Point
from sympy.core.symbol import symbols
from sympy import Symbol, Eq

def find_zeros(p: list):

  #*** this is the equation!
  #              0,5       1,6       2,7         3,8        4,9
  # d(x) = dt * (a_1 * x + b_1 * y + c_1 * x^2 + d_1 * x*y + e_1 * y^2) 
  # d(y) = dt * (a_2 * x + b_2 * y + c_2 * x^2 + d_2 * y*x + e_2 * y^2) 

  x, y = symbols('x y')
  Z = Point(x,y)
  r = solve([
             Z[0] * p[0] + Z[1] * p[1] + (Z[0]**2) * p[2] + (Z[0]**2) * p[4] + (Z[0]*Z[1]) * p[3],
             Z[0] * p[5] + Z[1] * p[6] + (Z[0]**2) * p[7] + (Z[1]**2) * p[9] + (Z[0]*Z[1]) * p[8],
                  ],Z,real=True)
  output = []
  for x in r:
    if x[0].is_real and x[1].is_real: output.append(x)
  if len(output)==1: output.append(output[0])
  return tuple(output)

def eval(p: list, Z: list):
  """
  auxiliary function for testing the roots that "find_zeros" finds!
  """
  return  (Z[0] * p[0] + Z[1] * p[1] + (Z[0]**2) * p[2] + (Z[0]**2) * p[4] + (Z[0]*Z[1]) * p[3],
             Z[0] * p[5] + Z[1] * p[6] + (Z[0]**2) * p[7] + (Z[1]**2) * p[9] + (Z[0]*Z[1]) * p[8])

def view(i):
  """
  auxiliary function for testing the roots that "find_zeros" finds!
  """
  try:
    with open('equations_and_solutions.pkl','rb') as w: data=pickle.load(w)
  except:
    with open('database/equations_and_solutions.pkl','rb') as w: data=pickle.load(w)
  k = list(data.keys())
  for j in range(len(data[k[i]])):
    print(f'root N {j} is {data[k[i]][j]} and evals to {eval(k[i],data[k[i]][j])}')


def generate_parameters(size: int = 10):
  return tuple(np.random.rand()*np.random.choice([-1,1],size))


def generator(TIMES):
  from datetime import datetime
  from dateutil.relativedelta import relativedelta
  data = {}
  ta = datetime.now()
  for _ in range(TIMES):
    if _%20==0: 
      tb = datetime.now()
      dt = relativedelta(tb,ta)
      print(f'Lap {_} of {TIMES} took {dt.seconds+dt.microseconds/1e6} seconds\n')
      ta = datetime.now()
    p = generate_parameters()
    z = find_zeros(p)
    data[tuple(p)] = z
  return data



<b> (2) Functions used to calculate the stability of the fixed points and make a veredict about the equation </b>
<br><i>(i.e. answer the question: does it have unstable equilibriums and hence do regions forbidden at infinite time exist?)</i>


In [8]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
import pickle

def matrix_calculator(p, x):
  """ 
  only for order 2!
  """
  #              0,5       1,6       2,7         3,8        4,9
  # d(x) = dt * (a_1 * x + b_1 * y + c_1 * x^2 + d_1 * x*y + e_1 * y^2) 
  # d(y) = dt * (a_2 * x + b_2 * y + c_2 * x^2 + d_2 * y*x + e_2 * y^2) 
  return    np.asarray([
              [  p[0]+2*p[2]*x[0]+p[3]*x[1],
                 p[1]+2*p[4]*x[1]+p[3]*x[0],],

              [  p[5]+2*p[7]*x[0]+p[8]*x[1],
                 p[6]+2*p[9]*x[1]+p[8]*x[0],],
                             ], dtype=np.float32)
                

def convergence_veredict(matrix):
  eigs = np.linalg.eigvals(matrix)
  realparts = []
  for x in eigs:
    if np.isreal(x): realparts.append(x)
    else: realparts.append(x.real)

  # (1) the method of x_n+1 = A * x_n 
  # and we want to guarantee that A
  # is an expansion
  # (2) the method of using that 
  # every eigensolution moves away
  # from the fixed point
  if False: 
    if min(realparts)>=1: return 1
  else:
    if max(realparts)>0: return 1
  return 0

def rule(p,x):
  return convergence_veredict(matrix_calculator(p,x))

<b>(3) connect to Google Drive and generate the database that our network will fit!</b>

In [ ]:
if True:

  # (0) connect to Google Drive
  from google.colab import drive
  drive.mount('/content/drive')
  import os
  try:
    os.mkdir('drive/My Drive/complex-dynamics')
  except: pass

  # (1) initialize
  data = {}
  names = [
           #'x0','y0',
              'a_1','b_1','c_1','d_1','e_1',
              'a_2','b_2','c_2','d_2','e_2',
                                       'divergence',]
  for x in names: data[x] = []

  # (2) generate equations and calculate their stationary points
  TIMES = 10000
  print(f'about to generate {TIMES} equations... will take approximately {TIMES//2} seconds')
  equations_and_solutions = generator(TIMES)

  # (3) append the convergence veredict for every case
  for k in equations_and_solutions.keys():
    counter = 0
    for j in equations_and_solutions[k]:        
      counter += rule(k, j)
    if counter>0: counter=1
    param = k + (float(counter),)
    for i,x in enumerate(names): data[x] += [param[i]]

  # (4) balance dataset
  df = pd.DataFrame(data)
  q = df['divergence'].value_counts()
  m = np.argmin([q[i] for i in range(2)])
  df = pd.concat([df[df['divergence']==m], df[~(df['divergence']==m)][:int(q[m]*1.25)]],0).sample(frac=1)

  # (5) save
  df.to_csv('drive/My Drive/complex-dynamics/database.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
about to generate 10000 equations... will take approximately 5000 seconds
Lap 0 of 10000 took 1.4e-05 seconds



In [ ]:
# THIS WILL KEEP COLLAB RUNNING 
# paste it in the browser's console; alternatively keep visiting the script every <30mins
#(max time= 12hs)
jsfunc="""
function ClickConnect(){
  colab.config
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End");
};
setInterval(ClickConnect, 1000000)
"""